In [41]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM,Qwen3ForCausalLM
# /share/public/public_models/Qwen3-14B/
# /share/others/public_models/Qwen3-14B/

tokenizer = AutoTokenizer.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-14B/")
model = AutoModelForCausalLM.from_pretrained("/inspire/hdd/global_public/public_models/Qwen/Qwen3-14B/")
model.half()

Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.41it/s]


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 5120)
    (layers): ModuleList(
      (0-39): 40 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=False)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=5120, out_features=17408, bias=False)
          (up_proj): Linear(in_features=5120, out_features=17408, bias=False)
          (down_proj): Linear(in_features=17408, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((5120,), eps=1e-06)
        (post_attention_layernorm): 

In [ ]:
import json
with open("./train_data/vicuna-bench_Qwen3-14B_data_0_2.json", "r") as f:
    data = json.load(f)
    

In [24]:
len(data['1']['Token'])

9

In [33]:
import torch
model_name = 'Qwen3-14B'
dataset = 'vicuna-bench'
begin = '0'
end = '2'
cur_hidden_states = torch.load(f'./train_data_backup/{dataset}_{model_name}_cur_hidden_states_{begin}_{end}.pt')
fake_last_hidden_states = torch.load(f'./train_data_backup/{dataset}_{model_name}_fake_last_hidden_states_{begin}_{end}.pt')
true_last_hidden_states = torch.load(f'./train_data_backup/{dataset}_{model_name}_true_last_hidden_states_{begin}_{end}.pt')

In [34]:
cur_hidden_states.shape

torch.Size([18, 1, 5120])

In [35]:
fake_last_hidden_states.shape

torch.Size([18, 1, 5120])

In [36]:
true_last_hidden_states.shape

torch.Size([20, 1, 5120])

In [29]:
import json
with open(f"./train_data/{dataset}_{model_name}_data_{begin}_{end}.json", "r") as f:
    data = json.load(f)

In [32]:
for key, value in data.items():
    print(len(value['Token']))

999
1517
1756
2182
2402
2563
3020
3726
3981
4175
4497
4767
4953
5203
5793
6034
6140
6429
6652
7029
7639
8114
8633
9423
9782
9993
10940
11541
11840
12369
12578
12779
13297
13854
14006
14414
14600
14789
15085
15501
16022
16598
17008
17492
17937
18636
19045
19433
19606
20207
20685
21236
21488
21927
22466
23465
23598
23733
24058
24282
24469
24609
24808
25265
25536
25720
26233
26510
27000
27465
27798
28062
28314
28563
28741
29110
29503
29883
30314
30580


In [47]:
fake_logits = model.lm_head(fake_last_hidden_states[12])
true_logits = model.lm_head(true_last_hidden_states[14])
fake_token = torch.argmax(fake_logits, dim=-1)
true_token = torch.argmax(true_logits, dim=-1)
print(fake_token, true_token)

tensor([279]) tensor([279])


In [23]:
print(model)

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): 

In [2]:
messages = [
    {"role": "system",
        "content": "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."},
]
messages.append({
            "role": "user",
            "content": "How are you"
        })
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
)

In [7]:
inputs['input_ids'].shape

torch.Size([1, 117])

In [15]:
import torch
min_dtype = torch.finfo(torch.float16).min
mask = torch.full((1, 117), fill_value=min_dtype, dtype=torch.float16)
cache_pos = torch.arange(117,118)
dag = torch.arange(117) > cache_pos.reshape(-1,1)

In [16]:
dag

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False]])

In [15]:
import json

with open("/home/xujiaming/xujiaming/Paper/SpecMoD/data/alpaca_Qwen3-8B_data.json", "r") as f:
    data = json.load(f)
    lst = []
    for key, value in data.items():
        lst.append(value['avg_len'])
    print(sum(lst) / len(lst))

12.49907672039015


In [16]:
lst = []
for key, value in data.items():
    lst.append(value['avg_len'])
print(sum(lst) / len(lst))

12.49907672039015


In [44]:
with open("/home/xujiaming/xujiaming/Paper/SpecMoD/data/alpaca_Qwen3-8B_data.json", "r") as f:
    data = json.load(f)
    lst = []
    ground_truth = []
    for q_id, q_re in data.items():
        for token_id, token_value in q_re['Token'].items():
            lst.append(eval(token_id))
            ground_truth.append(token_value['layer_index'])

In [32]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from model.qwen3_model import Spec_Qwen3ForCausalLM

model = Spec_Qwen3ForCausalLM.from_pretrained(f"/share/others/public_models/Qwen3-14B/")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 8/8 [00:35<00:00,  4.39s/it]


In [49]:
print(model.config)

Qwen3Config {
  "_attn_implementation_autoset": true,
  "architectures": [
    "Qwen3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 5120,
  "initializer_range": 0.02,
  "intermediate_size": 17408,
  "max_position_embeddings": 40960,
  "max_window_layers": 40,
  "model_type": "qwen3",
  "num_attention_heads": 40,
  "num_hidden_layers": 40,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000,
  "sliding_window": null,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.1",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}



In [46]:
import torch
data_emb = model.model.embed_tokens(torch.tensor(lst)).detach()
print()

In [47]:
data_emb.shape

torch.Size([237648, 5120])

In [18]:
questions[0]

{'question_id': 81,
 'category': 'writing',
 'turns': ['Compose an engaging travel blog post about a recent trip to Hawaii, highlighting cultural experiences and must-see attractions.',
  'Rewrite your previous response. Start every sentence with the letter A.']}

In [ ]:
a = [[[] for i in range(10)] for _ in range(10)]

In [ ]:
a={}

In [2]:
a[100] = 10

In [3]:
a

{100: 10}

In [60]:
a[0][0].append(5)

In [63]:
a

[[[5], [5], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []],
 [[], [], [], [], [], [], [], [], [], []]]

In [62]:
a[0][1] = a[0][0]

In [27]:
import torch
import torch.nn.functional as F

v1 = torch.tensor([1., 2., 3.]).unsqueeze(0).unsqueeze(0)
v2 = torch.tensor([1., 1., 3.]).unsqueeze(0).unsqueeze(0)

# 对于 1D 向量，我们需要在 dim=0 上计算
similarity_1d = F.cosine_similarity(v1, v2, dim= -1).mean()
print(f"v1: {v1}")
print(f"v2: {v2}")
print(f"1D 向量余弦相似度: {similarity_1d}")

v1: tensor([[[1., 2., 3.]]])
v2: tensor([[[1., 1., 3.]]])
1D 向量余弦相似度: 0.9669874906539917


In [12]:
import torch
a = torch.tensor([[1.,2.,3.]])

In [14]:
torch.argmax(a).item()

2

{'81': {'Prompt': [151644, 8948, 198, 2610, 525, 264, 10950, 11, 48050, 323, 10745, 17847, 13, 23240, 4226, 438, 1492, 3641, 438, 3204, 11, 1393, 1660, 6092, 13, 220, 4615, 11253, 1265, 537, 2924, 894, 27756, 11, 88635, 11, 24207, 11, 63782, 11, 20836, 11, 11406, 11, 476, 11816, 2213, 13, 5209, 5978, 429, 697, 14507, 525, 39318, 73215, 323, 6785, 304, 6993, 382, 2679, 264, 3405, 1558, 537, 1281, 894, 5530, 11, 476, 374, 537, 2097, 1832, 55787, 11, 10339, 3170, 4518, 315, 35764, 2494, 537, 4396, 13, 1416, 498, 1513, 944, 1414, 279, 4226, 311, 264, 3405, 11, 4486, 1513, 944, 4332, 895, 1995, 13, 151645, 198, 151644, 872, 198, 70492, 458, 22570, 5821, 5010, 1736, 911, 264, 3213, 8411, 311, 27521, 11, 38586, 12752, 11449, 323, 1969, 12, 4060, 38491, 13, 151645, 198, 151644, 77091, 198], 'Token': {'198': {'layer_index': [19, 20, 29, 30, 35, 36, 37, 39], 'similarity': 0.11242804676294327}, '32313': {'layer_index': [0, 1, 5, 8, 9, 11, 12, 13, 16, 21, 26, 28, 29, 30, 38, 39], 'similarity': 0.3

In [50]:
import torch.nn as nn
class PathPredictorMLP(nn.Module):
    def __init__(self, n_layers, llm_hidden_dim, mlp_internal_dim):
        super().__init__()
        
        self.input_dim = llm_hidden_dim 
        self.output_dim = n_layers
        
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, mlp_internal_dim),
            nn.ReLU(),
            nn.LayerNorm(mlp_internal_dim), 
            nn.Linear(mlp_internal_dim, self.output_dim)
        )
    
    def forward(self, x):
        return self.net(x)
predictor = PathPredictorMLP(
    n_layers=36,
    llm_hidden_dim=4096,
    mlp_internal_dim=1024,)
predictor.load_state_dict(torch.load("/home/xujiaming/xujiaming/Paper/SpecMoD/path_predictor_mlp_baseline.pth"))


<All keys matched successfully>

In [52]:
predictor

PathPredictorMLP(
  (net): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): ReLU()
    (2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (3): Linear(in_features=1024, out_features=36, bias=True)
  )
)

In [54]:
a = torch.randn(1, 1, 4096)
logits = predictor(a)

In [55]:
logits > 0

tensor([[[ True, False,  True, False,  True, False,  True, False,  True, False,
          False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False,  True, False, False, False,
          False,  True, False, False,  True,  True]]])

In [4]:
import torch
from model.EAGLE_model import Model


Ori_model_path = "/share/others/public_models/Qwen3-8B"
Spec_model_path = "/home/xujiaming/xujiaming/models/qwen3_8b_eagle3"
model = Model.from_pretrained(Spec_model_path=Spec_model_path, Ori_model_path=Ori_model_path, dtype=torch.float16, device=torch.device('cuda'))

In [5]:
model

Model(
  (embed_tokens): Embedding(151936, 4096)
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
  (midlayer): LlamaDecoderLayeremb(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=8192, out_features=4096, bias=False)
      (k_proj): Linear(in_features=8192, out_features=1024, bias=False)
      (v_proj): Linear(in_features=8192, out_features=1024, bias=False)
      (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
      (rotary_emb): LlamaRotaryEmbedding()
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
      (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
      (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
      (act_fn): SiLU()
    )
    (hidden_norm): LlamaRMSNorm()
    (input_layernorm): LlamaRMSNorm()
    (post_attention_layernorm): LlamaRMSNorm()
  )
  (fc): Linear(in_features=12288, out_features=4096, bias=False)
  (norm): L